In [1]:
# 导入同花顺包和登录同花顺
import datetime as dt
import pandas as pd
from numpy import *
from iFinDPy import *
THS_iFinDLogin("gdzq6888", "199212")

C:\Users\Xuwei\anaconda3\Lib\site-packages\iFinDPy.pth


0

In [2]:
from datetime import datetime
from calendar import monthrange
import numpy as np

In [3]:
# 定义给stock code分组的方程
def split(list_a, chunk_size):
    for i in range(0, len(list_a), chunk_size):
        yield list_a[i:i + chunk_size]

In [8]:
# 定义计算相关性平均值的方程
def find_upper_triangle(matrix):
    mask = np.triu(np.ones(matrix.shape), k=1)
    return matrix[mask.astype(bool)]

def solve(a):
    return find_upper_triangle(a)

# 多头alpha

## 个股截面离散度

In [28]:
# 输入日期格式为’年-月‘
def individual_stock_dispersion(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]
    # 获取中证800指数成分
    data = THS_DP('index_weight',month_end+';000906.SH','date:Y,thscode:Y,security_name:Y,weight:Y').data
    # 提取股票代码
    stock_code = list(data['thscode'])
    # 将股票代码分为4份
    stock_code2 = list(split(stock_code,200))
    #获得成分股每日价格
    data = pd.DataFrame()
    for i in stock_code2:
        a = THS_HQ(i,'close','',month_end_10years,month_end).data
        data=pd.concat([data,a])
    # 加入月，日
    data['year'] = data['time'].apply(lambda x: x.split('-')[0])
    data['month'] = data['time'].apply(lambda x: x.split('-')[1])
    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()

    # 计算日标准差
    std = data.groupby(['time'])['return'].std().reset_index()
    std = std.rename(columns={'return':'daily std'})
    std['month'] = std['time'].str[:7]

    # 得到离散度
    std_monthly = std.groupby('month')['daily std'].mean().reset_index()
    std_monthly = std_monthly.rename(columns={'daily std':'monthly std'})

    # 计算高/低离散度
    dispersion = list(std_monthly['monthly std'])[-1]/mean(list(std_monthly['monthly std']))

    return dispersion


In [30]:
individual_stock_dispersion('2023-01')

0.7584548441638147

## 行业截面离散度

In [10]:
def industry_dispersion(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]

    # 获取一级行业指数回报
    data=THS_HQ('CI005001.CI,CI005002.CI,CI005003.CI,CI005004.CI,CI005005.CI,CI005006.CI,CI005007.CI,CI005008.CI,CI005009.CI,CI005010.CI,CI005011.CI,CI005012.CI,CI005013.CI,CI005014.CI,CI005015.CI,CI005016.CI,CI005017.CI,CI005018.CI,CI005019.CI,CI005020.CI,CI005021.CI,CI005022.CI,CI005023.CI,CI005024.CI,CI005025.CI,CI005026.CI,CI005027.CI,CI005028.CI,CI005029.CI,CI005030.CI','close','',month_end_10years,month_end).data

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()

    # 计算日标准差
    std = data.groupby(['time'])['return'].std().reset_index()
    std = std.rename(columns={'return':'daily std'})
    std['month'] = std['time'].str[:7]

    # 得到离散度
    std_monthly = std.groupby('month')['daily std'].mean().reset_index()
    std_monthly = std_monthly.rename(columns={'daily std':'monthly std'})

    # 计算高/低离散度
    dispersion = list(std_monthly['monthly std'])[-1]/mean(list(std_monthly['monthly std']))

    return  dispersion


In [11]:
industry_dispersion('2023-01')

0.9762148769812952

## 个股相关性

In [12]:
# 输入日期格式为’年-月‘
def stock_correlation(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]
    # 获取中证800指数成分
    data = THS_DP('index_weight',month_end+';000906.SH','date:Y,thscode:Y,security_name:Y,weight:Y').data
    # 提取股票代码
    stock_code = list(data['thscode'])
    # 将股票代码分为4份
    stock_code2 = list(split(stock_code,200))
    #获得成分股每日价格
    data = pd.DataFrame()
    for i in stock_code2:
        a = THS_HQ(i,'close','',month_end_10years,month_end).data
        data=pd.concat([data,a])

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()
    data['month'] = data['time'].str[:7]

    # 计算相关性
    corr_list=list()
    for k in list(unique(data['month'])):
        df = pd.DataFrame()
        test = data[data['month']==k]
        for i in list(unique(test['thscode'])):
            a = test[test['thscode'] == i].reset_index()
            a = a[['return']]
            a = a.rename(columns={'return':i})
            df = pd.concat([df,a],axis=1)
        corr_matrix = df.corr().fillna(0).to_numpy()
        mean_correlation = mean(solve(corr_matrix))
        corr_list.append(mean_correlation)
    correlation_mean = mean(corr_list)

    correlation = corr_list[-1]/correlation_mean

    return correlation



In [13]:
stock_correlation('2023-01')

0.3746244164380103

## 行业相关性

In [16]:
def industry_correlation(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]

    # 获取一级行业指数回报
    data=THS_HQ('CI005001.CI,CI005002.CI,CI005003.CI,CI005004.CI,CI005005.CI,CI005006.CI,CI005007.CI,CI005008.CI,CI005009.CI,CI005010.CI,CI005011.CI,CI005012.CI,CI005013.CI,CI005014.CI,CI005015.CI,CI005016.CI,CI005017.CI,CI005018.CI,CI005019.CI,CI005020.CI,CI005021.CI,CI005022.CI,CI005023.CI,CI005024.CI,CI005025.CI,CI005026.CI,CI005027.CI,CI005028.CI,CI005029.CI,CI005030.CI','close','',month_end_10years,month_end).data

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()
    data['month'] = data['time'].str[:7]

    corr_list=list()
    for k in list(unique(data['month'])):
        df = pd.DataFrame()
        test = data[data['month']==k]
        for i in list(unique(test['thscode'])):
            a = test[test['thscode'] == i].reset_index()
            a = a[['return']]
            a = a.rename(columns={'return':i})
            df = pd.concat([df,a],axis=1)
        corr_matrix = df.corr().fillna(0).to_numpy()
        mean_correlation = mean(solve(corr_matrix))
        corr_list.append(mean_correlation)
    correlation_mean = mean(corr_list)

    correlation = corr_list[-1]/correlation_mean

    return correlation

In [18]:
industry_correlation('2023-01')

0.3376035150682084

## 市场流动性